In [5]:
import pandas as pd
import os
import numpy as np

In [ ]:
[os.remove(os.path.join(".","gamesRaw",str(x))) for x in os.listdir("gamesRaw") if x[-4:] == ".txt"]

In [6]:
# df = pd.read_csv("games.csv", usecols = ['winner', 'opening_eco', 'opening_ply'])
# df["winner"].replace({"white": 1, "black": 0, "draw": 0.5}, inplace=True)
# df["opening_eco_ori"] = df["opening_eco"]
# for i in range(len(df["opening_eco"].unique())):
#     df["opening_eco"].replace({df["opening_eco"].unique()[i]: i}, inplace=True)
# tresh = 50
# list_to_keep = df["opening_eco"].value_counts() >= tresh
# for i, j  in list_to_keep.iteritems():
#     if not j:
#         df = df.drop(df[df["opening_eco"] == i].index)
# df.to_csv("game_pre.csv")

In [12]:
#df = pd.read_csv("games.csv", usecols = ['winner', 'opening_eco', 'opening_ply'])

# Create Dataset

In [4]:
import requests
import json
players = []
for title in ["GM", "IM", "FM", "CM"]:
    r = requests.get('https://api.chess.com/pub/titled/{0}'.format(title))
    players = players + json.loads(str(r.json()).replace("\'","\""))["players"]
    print(title, len(players))

GM 1451
IM 3407
FM 6537
CM 7590


In [3]:
dict_keys(['url', 'move_by','time_control', 'last_activity', 'rated', 'turn', 'fen', 'start_time', 'time_class', 'rules', 'white', 'black', 'tournament'])
1 : white win, black lose
-1 : black win, white lose
0 : draw


SyntaxError: illegal target for annotation (1571951862.py, line 2)

In [104]:
dico_col = {'a': 7, 'b': 6, 'c': 5, 'd': 4, 'e': 3, 'f': 2, 'g': 1, 'h': 0}

In [5]:
import chess.pgn
import io
game_number = 0

for player in players:
    r = requests.get("https://api.chess.com/pub/player/{0}/games/2022/08".format(player))
    games = r.json()["games"]
    if games != []:
        for game in games:
            if "end_time" in game.keys() and game["rated"]:
                game_number +=1 
                if game["white"]["result"] == "win":
                    won = 1
                elif game["black"]["result"] == "win":
                    won = -1
                else:
                    won = 0
                if "pgn" in game.keys():
                    pgn = io.StringIO(game["pgn"])
                    gamepgn = chess.pgn.read_game(pgn)
                    board = gamepgn.board()
                    plate_number = 0
                    with open(os.path.join("gamesRaw", str(game_number) + ".txt"), "w") as f:
                        json_tmp = {"won": str(won), "boards": {}}
                        for move in gamepgn.mainline_moves():
                            board.push(move)
                            plate_number += 1
                            if plate_number >= 10:
                                boardstr = str(board).replace("p","1").replace("r","5").replace("n","3,0000001").replace("b", "3").replace("q", "8").replace("k", "2")
                                boardstr = boardstr.replace("P","-1").replace("R","-5").replace("N","-3,0000001").replace("B", "-3").replace("Q", "-8").replace("K", "-2")
                                boardstr = boardstr.replace(".","0")
                                json_tmp["boards"][plate_number] = str(boardstr)
                        json.dump(json_tmp, f)
print(game_number)

427641


# Model

In [10]:
import keras
from random import shuffle

2022-09-18 20:56:39.735802: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-18 20:56:39.735867: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-18 20:56:40.431013: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-18 20:56:51.713072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-18 20:56:51.713283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [11]:
model = keras.Sequential()
model.add(keras.Input(shape=(64)))
model.add(keras.layers.Dense(64))
model.add(keras.layers.Dense(8))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(  optimizer=keras.optimizers.Adam(learning_rate=0.5),
                loss=keras.losses.CategoricalCrossentropy(),
                metrics=[keras.metrics.CategoricalAccuracy()])
model.summary()                  

2022-09-18 20:57:01.577601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-18 20:57:01.591244: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-18 20:57:01.591310: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sotsaka-HP): /proc/driver/nvidia/version does not exist


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 8)                 520       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 4,689
Trainable params: 4,689
Non-trainable params: 0
_________________________________________________________________


In [28]:
print(len(l))

34


In [31]:
print(np.shape(m))

(34, 64)


# Load data

In [1]:
import os
import json
import sklearn
import numpy as np
from alive_progress import alive_bar

In [2]:
files = [x for x in os.listdir("gamesRaw") if x[-4:] == ".txt"]

In [3]:
print(len(files))

420855


In [4]:
def file_to_matrix(filename):
    content = json.load(open(filename))
    liste_de_matrix = []
    labels = []
    for mat in content['boards'].values():
        z = np.zeros(shape=(8,8))
        for i in range(len(mat.split("\n"))):
            for j in range(len(mat.split("\n")[i].split(" "))):
                z[i,j] = float(mat.split("\n")[i].split(" ")[j].replace(",","."))
        liste_de_matrix.append(z.flatten())
        labels.append(np.array([float(content['won'])]))
    return np.array(liste_de_matrix), np.array(labels)

In [5]:
l = 0
files = [x for x in os.listdir("gamesRaw") if x[-4:] == ".txt"]
lenfiles = len(files)
for file in files:
    l+=1
    print(np.floor(l*1000/lenfiles)/10, "%", end="\r")

In [5]:
matrixs = []
labels = []
l = 0
lenfiles = len(files)
for file in sorted(files)[:50000]:
    matrixs_tmp, labels_tmp = file_to_matrix(os.path.join("gamesRaw", file))
    [matrixs.append(mat) for mat in matrixs_tmp]
    [labels.append(mat) for mat in labels_tmp]
    l+=1
    print(np.floor(l*1000/lenfiles)/10, "%", end="\r")

AttributeError: module 'sklearn' has no attribute 'model_selection'

In [6]:
matrixs = np.array(matrixs)
labels = np.array(labels)

In [8]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest = train_test_split(matrixs, test_size=2/9, train_size=7/9, shuffle=False)
Ytrain, Ytest = train_test_split(labels, test_size=2/9, train_size=7/9, shuffle=False)

In [ ]:
model.fit(x = Xtrain, y=Ytrain, epochs=10)

2022-09-18 20:57:03.272929: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 731299840 exceeds 10% of free system memory.


Epoch 1/10


2022-09-18 20:57:14.510581: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22853120 exceeds 10% of free system memory.
2022-09-18 20:57:14.548774: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22853120 exceeds 10% of free system memory.
2022-09-18 20:57:16.995036: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22853120 exceeds 10% of free system memory.
2022-09-18 20:57:17.014744: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22853120 exceeds 10% of free system memory.


89270/89270 [==============================] - 96s 1ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 2/10
89270/89270 [==============================] - 94s 1ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 3/10
33083/89270 [==========>...................] - ETA: 58s - loss: 0.0000e+00 - categorical_accuracy: 1.0000

In [75]:
from sklearn.metrics import accuracy_score
y = model.predict(Xtest)
accuracy_score(y, Ytest)

4/4 [==============================] - 0s 2ms/step


(array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      